# Loading Libraries

In [7]:
from urllib.request import urlopen
from urllib.parse import urlparse
import urllib
from bs4 import BeautifulSoup
import requests
import pandas as pd
import sys
import time
import requests
from lxml import html
import sys
import os
from google.colab import drive


# Scraping all available game urls from Steam

In [ ]:
urlss=[]
opener = urllib.request.build_opener()
opener.addheaders = [('User-agent', 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.102 Safari/537.36')]
urllib.request.install_opener(opener)
url='https://store.steampowered.com/search/?sort_by=&sort_order=0&page='
for i in range(3240):
  try:
    response = urllib.request.urlopen(url+str(i))
    soup = BeautifulSoup(response, 'html.parser')
    raw_urls = soup.find('div', attrs={'id': 'search_resultsRows'})
    raw_urls = raw_urls.find_all('a')
    for j in raw_urls:
      urlss.append(j['href'])
    print(url+str(i))
  except:
    print(url+str(i))
    print(str(sys.exc_info()))
df = pd.DataFrame(urlss,columns =['urls'])
df.to_csv('urls.csv',index=False)

In [8]:
# urlss=pd.read_csv('urls.csv')
urll='https://raw.githubusercontent.com/junkuse/amzrepo/master/urls.csv'
urlss=pd.read_csv(urll)
urlss=list(urlss['urls'])

In [9]:
len(urlss)

81025

# Scraping All Game Data from Scraped URLS

In [10]:
filter = ''.join([chr(i) for i in range(1, 32)])
cookies = {'birthtime': '568022401'}
def cleantext(xpathh):
  try:
    attr = tree.xpath(xpathh)
    attr = attr[0].text_content()
    attr = attr.translate(str.maketrans('', '', filter))
    return str(attr)
  except:
    # print(str(xpathh)+' '+str(sys.exc_info()))
    return '-'

In [11]:
games = pd.DataFrame(columns=['url','name', 'categories','img_url','user_reviews', 'all_reviews',\
                              'date', 'developer', 'publisher', 'price', 'requirements', 'popu_tags',\
                              'desc', 'full_desc', 'pegi', 'pegi_url'])

In [ ]:
count=0
for i in urlss:
  try:
    print(str(count)+' '+i)
    r = requests.get(i, cookies=cookies)
    tree = html.fromstring(r.content)
    try:
      name = tree.xpath("//div[@class='apphub_AppName']")
      name=name[0].text_content()
    except:
      name='-'
    categories = cleantext("//div[@id='category_block']")
    desc = cleantext("//div[@class='game_description_snippet']")
    usr_rev = cleantext("//div[@class='user_reviews']//div[1]//div[2]")
    all_rev = cleantext("//div[@class='rightcol']//div[2]//div[2]")
    date = cleantext("//div[@class='date']")
    dev = cleantext("//div[@id='developers_list']")
    pub = cleantext("//div[@class='block']//div[5]//div[2]//a[1]")
    purchase = cleantext("//div[contains(@class,'game_area_purchase_game')]")
    sys_req = cleantext("//div[@class='game_page_autocollapse sys_req']")
    pop_tags = cleantext("//div[@class='glance_tags popular_tags']")
    try:
      game_img = tree.xpath("//img[@class='game_header_image_full']")
      game_img=game_img[0].get('src')
    except:
      game_img='-'
    game_full_desc = cleantext("//div[@id='game_area_description']")
    pegi = cleantext("//div[@class='game_rating_details']")
    try:
      soup = BeautifulSoup(str(r.content), 'html.parser')
      raw_urls = soup.find('div', attrs={'class': 'game_rating_icon'})
      raw_urls = raw_urls.find('img')
      pegi_url=raw_urls['src']  
    except:
      pegi_url='-'
    listt=[i,name,categories,game_img,usr_rev,all_rev,date,
           dev,pub,purchase,sys_req,pop_tags,desc,
           game_full_desc,pegi,pegi_url]
    listt = pd.Series(listt, index = games.columns)
    games = games.append(listt, ignore_index=True)
    count=count+1
  except:
    pass

# Finding Number of Unavailable Data

In [13]:
for i in games.columns:
  print(i)
  print(list(games[i]).count('-'))

url
0
name
1594
categories
2208
img_url
1594
user_reviews
1594
all_reviews
1595
date
1695
developer
1744
publisher
26980
price
336
requirements
2131
popu_tags
1875
desc
14462
full_desc
602
pegi
20270
pegi_url
20270


# Finding rows with more number of Unavailabe Data

In [15]:
a=games
c=0
for ind in a.index: 
  x=0
  for j in a.columns:
     if (a[j][ind] != '-'):
       x=x+1
  if (x>13):
    c=c+1
c

6480

# Exporting as CSV and storing it in Google Drive

In [17]:
games.to_csv('games1.csv',index=False)
drive.mount('/gdrive', force_remount=True)
with open('/gdrive/My Drive/games1.csv', 'w') as f:
  games.to_csv(f,index=False)

Mounted at /gdrive


# File Size Statistics

In [19]:
file_stats = os.stat('games1.csv')
print(file_stats)
print(f'File Size in Bytes is {file_stats.st_size}')
print(f'File Size in MegaBytes is {file_stats.st_size / (1024 * 1024)}')

os.stat_result(st_mode=33188, st_ino=5111985, st_dev=51, st_nlink=1, st_uid=0, st_gid=0, st_size=52923720, st_atime=1602903514, st_mtime=1602903515, st_ctime=1602903515)
File Size in Bytes is 52923720
File Size in MegaBytes is 50.47199249267578
